In [2]:
import modin.pandas as pd
import os
import numpy as np

import ray

In [3]:
import tensorflow as tf
import tensorflow_ranking as tfr
from tensorflow_serving.apis import input_pb2
from sklearn.datasets import load_svmlight_file

2024-06-04 10:22:43.565854: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-04 10:22:43.631514: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-04 10:22:43.632785: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 10:22:45.129234: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
folder = 1

df_train = pd.read_csv(f'MLSR/Fold{folder}/train.txt', sep=' ', header=None)

In [5]:
df_train[:10]

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
0,2,qid:1,1:3,2:3,3:0,4:0,5:3,6:1,7:1,8:0,...,128:11089534,129:2,130:116,131:64034,132:13,133:3,134:0,135:0,136:0,NaN
1,2,qid:1,1:3,2:0,3:3,4:0,5:3,6:1,7:0,8:1,...,128:11089534,129:2,130:124,131:64034,132:1,133:2,134:0,135:0,136:0,NaN
2,0,qid:1,1:3,2:0,3:2,4:0,5:3,6:1,7:0,8:0.666667,...,128:3,129:1,130:124,131:3344,132:14,133:67,134:0,135:0,136:0,NaN
3,2,qid:1,1:3,2:0,3:3,4:0,5:3,6:1,7:0,8:1,...,128:11089534,129:13,130:123,131:63933,132:1,133:3,134:0,135:0,136:0,NaN
4,1,qid:1,1:3,2:0,3:3,4:0,5:3,6:1,7:0,8:1,...,128:5,129:7,130:256,131:49697,132:1,133:13,134:0,135:0,136:0,NaN
5,1,qid:1,1:3,2:0,3:3,4:0,5:3,6:1,7:0,8:1,...,128:6,129:7,130:210,131:49923,132:5,133:15,134:0,135:0,136:0,NaN
6,1,qid:1,1:3,2:0,3:3,4:0,5:3,6:1,7:0,8:1,...,128:6,129:7,130:256,131:50023,132:3,133:14,134:0,135:0,136:0,NaN
7,2,qid:1,1:3,2:0,3:3,4:0,5:3,6:1,7:0,8:1,...,128:0,129:17,130:115,131:63318,132:1,133:3,134:0,135:0,136:0,NaN
8,1,qid:1,1:3,2:0,3:3,4:0,5:3,6:1,7:0,8:1,...,128:11089534,129:15,130:124,131:63363,132:1,133:3,134:0,135:0,136:0,NaN
9,0,qid:1,1:3,2:0,3:3,4:0,5:3,6:1,7:0,8:1,...,128:1,129:1,130:541,131:63363,132:1,133:5,134:0,135:0,136:0,NaN


## Format the data

In [4]:
def preprocess_features():
    '''This function processes and creates our feature columns descriptions'''
    # Read in the features file
    features = pd.read_csv('MLSR/features.csv')
    # Create new header and replace spaces with underscore
    new_header = features.iloc[0].str.replace(' ','_')
    # Remove the first row which is now the new header
    features = features[1:]
    # Set new headers
    features.columns = new_header
    # Only the first cell for each category is filled. Using forward will
    # will allow me to map each category to their sub-categories located
    # in the stream column 
    features['feature_description'] = features['feature_description'].ffill()
    # Replacing characters to allign with TensorFlows regex requirements
    character_removal = [' ', '(', ')', '*']
    for char in character_removal:
        features['feature_description'] = features['feature_description'].str.replace(char, '_')
        features['stream'] = features['stream'].astype(str).str.replace(char, '_')
    # Setting column type to string for mapping within the load_rename_save function
    features['feature_id'] = features['feature_id'].astype(str)
    # Creating new column to map features to existing dataset
    features['cols'] = 'string'
    # Looping over all features and creating new column name
    for idx in range(len(features)):
        if str(features.iloc[idx]['stream']) != 'nan':
            features['cols'].iloc[idx] = features['feature_description'].iloc[idx] + '_' + features['stream'].iloc[idx]
        else:
            features['cols'].iloc[idx] = features['feature_description'].iloc[idx]
    return features

In [5]:
def label_columns(df):
    '''This function labels the columns by descriptions
       found on the microsoft research page'''    
        
    for col in df.columns:
        if col == 0:
            df.rename({col : 'relevance_label'}, axis=1, inplace=True)
        elif col == 1:
            df.rename({col : 'query_id'}, axis=1, inplace=True)
        else:
            df.rename({col : f'feature_{col - 1}'}, axis=1, inplace=True)
            
    return df

In [6]:
def load_rename_save(folder_num):
    '''This function reads in all data located in folder n,
       labels the columns, removes uneeded elements from the cells (i.e. 'qid:1' the qid is uneeded),
       and saves the files as a parquet within folder n'''
    
    for folder in range(1, folder_num+1):
        # Load data
        df_train = pd.read_csv(f'MLSR/Fold{folder}/train.txt', sep=' ', header=None)
        df_test = pd.read_csv(f'MLSR/Fold{folder}/test.txt', sep=' ', header=None)
        df_val = pd.read_csv(f'MLSR/Fold{folder}/vali.txt', sep=' ', header=None)
        
        # Label the columns
        df_train = label_columns(df_train)
        df_test = label_columns(df_test)
        df_val = label_columns(df_val)
        
        # Remove 'n:' from each column. The dataset assigned each feature number
        # to the cells value which needs to be removed to get the data into int/float format
        dataframes = {'train': df_train, 'test': df_test, 'val': df_val}
        for k, df in dataframes.items():
            for i in range(1,len(df.columns)-1):
                df[f'feature_{i}'].replace(f'{i}:', '', regex=True, inplace=True)          
            
        # Only query_id was different than all of the other columns when assigning 
        # the prefix to the values. Here we remove 'qid:' from each cell
            df['query_id'].replace('qid:', '', regex=True, inplace=True)

        # Rename the feature columns from the given descriptions on Microsofts webiste   
        features = preprocess_features()
        
        for k, df in dataframes.items():
            for idx in range(len(features)):
                id_ = features.iloc[idx]['feature_id']
                for col in df.columns:
                    if str(id_) == col.lstrip('feature_'):
                        df.rename({col: features.iloc[idx]['cols']}, axis=1, inplace=True)
        
        # Save the cleaned dataset as a csv
        df_train.to_csv(f'MLSR/Fold{folder}/df_train.csv', index=False)
        df_test.to_csv(f'MLSR/Fold{folder}/df_test.csv', index=False)
        df_val.to_csv(f'MLSR/Fold{folder}/df_val.csv', index=False)

In [15]:
load_rename_save(5)

In [16]:
tmp = pd.read_csv(f'MLSR/Fold1/df_train.csv', index_col=0)
display(tmp)

,query_id,covered_query_term_number_body,covered_query_term_number_anchor,covered_query_term_number_title,covered_query_term_number_url,covered_query_term_number_whole_document,covered_query_term_ratio_body,covered_query_term_ratio_anchor,covered_query_term_ratio_title,covered_query_term_ratio_url,...,Inlink_number,Outlink_number,PageRank,SiteRank,QualityScore,QualityScore2,Query-url_click_count,url_click_count,url_dwell_time,feature_137
relevance_label,,,,,,,,,,,,,,,,,,,,,
2,1,3,3,0,0,3,1.0,1.0,0.000000,0.0,...,11089534,2,116,64034,13,3,0,0,0.0,NaN
2,1,3,0,3,0,3,1.0,0.0,1.000000,0.0,...,11089534,2,124,64034,1,2,0,0,0.0,NaN
0,1,3,0,2,0,3,1.0,0.0,0.666667,0.0,...,3,1,124,3344,14,67,0,0,0.0,NaN
2,1,3,0,3,0,3,1.0,0.0,1.000000,0.0,...,11089534,13,123,63933,1,3,0,0,0.0,NaN
1,1,3,0,3,0,3,1.0,0.0,1.000000,0.0,...,5,7,256,49697,1,13,0,0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,29992,2,0,1,1,2,1.0,0.0,0.500000,0.5,...,7,2,13829,35302,21,8,0,0,0.0,NaN
0,29992,2,1,1,1,2,1.0,0.5,0.500000,0.5,...,0,0,26074,35101,14,7,1,0,0.0,NaN
0,29992,2,2,2,2,2,1.0,1.0,1.000000,1.0,...,11,2,2995,62170,4,8,0,0,0.0,NaN


## Verify Data Integrity

In [7]:
def data_stats(folder_num):
    ''' This function is to collect basic stats from the dataset. '''
    for folder in range(1, folder_num+1):
        # Load the data
        df_train = pd.read_csv(f'MLSR/Fold{folder}/df_train.csv')
        df_test = pd.read_csv(f'MLSR/Fold{folder}/df_test.csv')
        df_val = pd.read_csv(f'MLSR/Fold{folder}/df_val.csv')
        
        # Collect metrics for below stats
        len_train = len(df_train)
        len_test = len(df_test)
        len_val = len(df_val)
        total = len_train + len_test + len_val
        
        # Print length of all datasets and the overal balance between the splits
        print('*'*24 + ' ' + f'Folder Number {folder}' + ' ' + '*'*24)
        print(f'Total rows in training set {folder}: {len_train}')
        print(f'Total rows in testing set {folder}: {len_test}')
        print(f'Total rows in validation set {folder}: {len_val}')
        print('='*64)
        print(f'The training set contains {round(len_train/total, 2) * 100}% of the total data')
        print(f'The testing set contains {round(len_test/total, 2) * 100}% of the total data')
        print(f'The validation set contains {round(len_val/total, 2) * 100}% of the total data')
        print('='*64)
        
        # Create new dataframe showing NaN values
        df_train_ = pd.DataFrame(df_train.isna().sum(), columns=['NaN_values'])
        df_test_ = pd.DataFrame(df_test.isna().sum(), columns=['NaN_values'])
        df_val_ = pd.DataFrame(df_val.isna().sum(), columns=['NaN_values'])
        
        # Mapping of NaN dataframes
        nan_dataframes = {'df_train':df_train_, 'df_test':df_test_, 'df_val':df_val_}
        
        # Print the total percentage of missing values per column in each dataframe
        for k,df in nan_dataframes.items():
            df = df[df['NaN_values'] > 0]
            total_missing = [len(df.index)]
            for missing in total_missing:
                if k == 'df_train':
                    print(f'df_train_ Column {df.index[missing - 1]} is missing {df.values[missing - 1] / len(df_train) * 100}% of its data')
                elif k == 'df_test':
                    print(f'df_test_ Column {df.index[missing - 1]} is missing {df.values[missing - 1] / len(df_test) * 100}% of its data')
                else:
                    print(f'df_val_ Column {df.index[missing - 1]} is missing {df.values[missing - 1] / len(df_val) * 100}% of its data')
        
        # Mapping of initial dataframes
        dataframes = {'df_train': df_train, 'df_test': df_test, 'df_val': df_val}
        
        # Calculating the distribution of the relevance column
        for k,df in dataframes.items():
            df_len = len(df)
            relevance_counts = df['relevance_label'].value_counts()
            print('='*64)
            print('*'*16 + ' ' + f'{k} Relevance Class Balance' + ' ' + '*'*16)
            for i in [0,1,2,3,4]:
                print(f'Rank {i}: Total Count: {relevance_counts[i]} Percentage: {round(relevance_counts[i]/df_len,2) * 100}%')
        print(' ')

In [22]:
data_stats(5)

************************ Folder Number 1 ************************
Total rows in training set 1: 723412
Total rows in testing set 1: 241521
Total rows in validation set 1: 235259
The training set contains 60.0% of the total data
The testing set contains 20.0% of the total data
The validation set contains 20.0% of the total data
df_train_ Column feature_137 is missing [100.]% of its data
df_test_ Column feature_137 is missing [100.]% of its data
df_val_ Column feature_137 is missing [100.]% of its data
**************** df_train Relevance Class Balance ****************
Rank 0: Total Count: 377957 Percentage: 52.0%
Rank 1: Total Count: 232569 Percentage: 32.0%
Rank 2: Total Count: 95082 Percentage: 13.0%
Rank 3: Total Count: 12658 Percentage: 2.0%
Rank 4: Total Count: 5146 Percentage: 1.0%
**************** df_test Relevance Class Balance ****************
Rank 0: Total Count: 124784 Percentage: 52.0%
Rank 1: Total Count: 77896 Percentage: 32.0%
Rank 2: Total Count: 32459 Percentage: 13.0%
R

In [8]:
def drop_unwanted_cols(folder_num):
    '''This function drops column 137 from each dataframe due to 137 missing 
        100% of its values across all datasets'''
    for folder in range(1, folder_num+1):
        df_train = pd.read_csv(f'MLSR/Fold{folder}/df_train.csv')
        df_test = pd.read_csv(f'MLSR/Fold{folder}/df_test.csv')
        df_val = pd.read_csv(f'MLSR/Fold{folder}/df_val.csv')
        
        df_train.drop('feature_137', axis=1, inplace=True)
        df_test.drop('feature_137', axis=1, inplace=True)
        df_val.drop('feature_137', axis=1, inplace=True)
        
        df_train.to_csv(f'MLSR/Fold{folder}/df_train.csv', index=False)
        df_test.to_csv(f'MLSR/Fold{folder}/df_test.csv', index=False)
        df_val.to_csv(f'MLSR/Fold{folder}/df_val.csv', index=False)
        print(f'Finished Cleaning Fold{folder}')

In [24]:
drop_unwanted_cols(5)
tmp = pd.read_csv(f'MLSR/Fold1/df_train.csv', index_col=0)
display(tmp)

Finished Cleaning Fold1
Finished Cleaning Fold2
Finished Cleaning Fold3
Finished Cleaning Fold4
Finished Cleaning Fold5


,query_id,covered_query_term_number_body,covered_query_term_number_anchor,covered_query_term_number_title,covered_query_term_number_url,covered_query_term_number_whole_document,covered_query_term_ratio_body,covered_query_term_ratio_anchor,covered_query_term_ratio_title,covered_query_term_ratio_url,...,Length_of_URL,Inlink_number,Outlink_number,PageRank,SiteRank,QualityScore,QualityScore2,Query-url_click_count,url_click_count,url_dwell_time
relevance_label,,,,,,,,,,,,,,,,,,,,,
2,1,3,3,0,0,3,1.0,1.0,0.000000,0.0,...,62,11089534,2,116,64034,13,3,0,0,0.0
2,1,3,0,3,0,3,1.0,0.0,1.000000,0.0,...,54,11089534,2,124,64034,1,2,0,0,0.0
0,1,3,0,2,0,3,1.0,0.0,0.666667,0.0,...,45,3,1,124,3344,14,67,0,0,0.0
2,1,3,0,3,0,3,1.0,0.0,1.000000,0.0,...,56,11089534,13,123,63933,1,3,0,0,0.0
1,1,3,0,3,0,3,1.0,0.0,1.000000,0.0,...,64,5,7,256,49697,1,13,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,29992,2,0,1,1,2,1.0,0.0,0.500000,0.5,...,39,7,2,13829,35302,21,8,0,0,0.0
0,29992,2,1,1,1,2,1.0,0.5,0.500000,0.5,...,28,0,0,26074,35101,14,7,1,0,0.0
0,29992,2,2,2,2,2,1.0,1.0,1.000000,1.0,...,58,11,2,2995,62170,4,8,0,0,0.0


## Convert CSV to TFRecord

In [9]:
class SVM_to_TFRecord():
    '''SVM_to_TFRecord converts .txt files to TFRecords using sklearns load_svmlight_file which 
       loads the files in SVMlight format allowing faster computations. Loading .txt with this
       method returns three arrays when query_id is set to True. 
       
           Array 0 = Features
           Array 1 = Relevance
           Array 2 = Query ID
           
      SVM_to_TFRecord Args:
                          data_path: Path to load .txt data
                          tfrecord_path: Path to save .tfrecords           
           '''
    
    def __init__(self, data_path:str='', tfrecord_path:str=''):
        self.data_path = data_path
        self.tfrecord_path = tfrecord_path
        
        if not os.path.isdir(self.data_path):
            print('Please enter correct data path!!!!')

        if not os.path.isdir(self.tfrecord_path):
            os.makedirs(self.tfrecord_path)

    # Helper functions for creating TF Features
    # Note: There are only two feature types in 
    # the datasets
    def _float_feature(self, value):
      """Returns a float_list from a float / double."""
      return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

    def _int64_feature(self, value):
      """Returns an int64_list from a bool / enum / int / uint."""
      return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
    
    def create_tfrecord(self, data, tfrecord_path):
        
        options_ = tf.io.TFRecordOptions(compression_type='GZIP')
        with tf.io.TFRecordWriter(tfrecord_path) as writer:

            # Create Example list 
            elwc = input_pb2.ExampleListWithContext()
            # Save the last query id for filtering
            last_query_id = None
            # Map Feature names to dictionary
            features_ = preprocess_features()
            # Extract column names
            labels = np.array(features_['cols'])

            for row in range(data.shape[0]):
                # Select data from each row
                relevance_label, query_id, features = data[row,0], data[row,1], data[row,2:]
                # Create Example Dict
                example_dict = {
                    f'{feat_name}':self._float_feature(feat_val)
                    for feat_name, feat_val in zip(labels, features)
                }
                example_dict['relevance_label'] = self._int64_feature(int(relevance_label))
                # Create Features
                example_ = tf.train.Example(features=tf.train.Features(feature=example_dict))
                # Create ELWC by query id
                if query_id != last_query_id:
                    if last_query_id != None:
                        writer.write(elwc.SerializeToString())
                    last_query_id = query_id
                    elwc = input_pb2.ExampleListWithContext()
                    elwc.examples.append(example_)
                else:
                    elwc.examples.append(example_)
            # Writing the final query
            writer.write(elwc.SerializeToString())
            
    def load_and_convert_data(self):
        
        train = load_svmlight_file(f'{self.data_path}/train.txt', query_id=True)
        test = load_svmlight_file(f'{self.data_path}/test.txt', query_id=True)
        val = load_svmlight_file(f'{self.data_path}/vali.txt', query_id=True)
        
        # Note: train, test, val each return three numpy arrays:
            # Array 0 = features
            # Array 1 = relevance
            # Array 3 = Query IDences in a batch fit a given 
            
        data_ = {'train':train, 'test':test, 'val':val}
        
        # Concatenate data into one array
        for label, data in data_.items():
            if label == 'train':
                train = np.concatenate((np.expand_dims(train[1], axis=1), np.expand_dims(train[2], axis=1), train[0].toarray()), axis=1)
                tfrecord_path = f'{self.tfrecord_path}/train.tfrecords'
                self.create_tfrecord(train, tfrecord_path)
                
            elif label == 'test':
                test = np.concatenate((np.expand_dims(test[1], axis=1), np.expand_dims(test[2], axis=1), test[0].toarray()), axis=1)
                tfrecord_path = f'{self.tfrecord_path}/test.tfrecords'
                self.create_tfrecord(test, tfrecord_path)

            else:
                val = np.concatenate((np.expand_dims(val[1], axis=1), np.expand_dims(val[2], axis=1), val[0].toarray()), axis=1)
                tfrecord_path = f'{self.tfrecord_path}/val.tfrecords'
                self.create_tfrecord(val, tfrecord_path)

In [7]:
data_path = ['Fold1', 'Fold2', 'Fold3', 'Fold4', 'Fold5']

for data in data_path:
    path = f"tfrecord_{data.lstrip('Fold')}"
    converter = SVM_to_TFRecord(f"MLSR/{data}", path)
    converter.load_and_convert_data()
    print(f'Completed {data}/{path}')

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2024-06-03 09:55:43,527	INFO worker.py:1752 -- Started a local Ray instance.


Completed Fold1/tfrecord_1
Completed Fold2/tfrecord_2
Completed Fold3/tfrecord_3
Completed Fold4/tfrecord_4
Completed Fold5/tfrecord_5


In [10]:
def build_model(model_dir,
                train_input_pattern:str='tfrecord_1/train.tfrecords',
                valid_input_pattern:str='tfrecord_1/val.tfrecords',
                num_epochs:int=5,
                optimizer:str='adagrad',
                loss:str="approx_ndcg_loss",
                hidden_layer_dims:list=[1024,512,256],
                activation=tf.nn.relu,
                use_batch_norm:bool=True,
                batch_norm_moment:float=0.999,
                dropout:float=0.5,
                list_size:int=200,
                preprocessor:bool=False,
                train_batch_size:int=128,
                valid_batch_size:int=128,
                steps_per_epoch:int=5000,
                validation_steps:int=125,
                learning_rate:float=0.05):
    
    '''Trains and validates data via TensorFlowRanking Pipeline with DNNScorer
    
    Args:
        model_dir: Directory location to save modeling information (Note: Code will generate new folder)
        train_input_pattern: training data file location
        valid_input_pattern: validation data file location
        optimizer: tf.keras.optimizers
        num_epochs: total training epochs
        loss: model loss function --> default: 'approx_ndcg_loss'
        hidden_layer_dims: hidden layer dimensions
        activation: activation for each layer
        use_batch_norm: use of batch normalization for each layer
        batch_norm_moment: momentum for the moving average in batch normalization
        dropout: dropout rate
        preprocessor: use of preprosser (Note: Current implmentation = log1p)
        train_batch_size: training batch size
        valid_batch_size: validation batch size
        steps_per_epoch: number of training steps per epoch
        validation_steps: number of validation steps per epoch
        learning_rate: learning rate     
    '''
    
    # Define Scorer
    scorer = tfr.keras.model.DNNScorer(hidden_layer_dims=hidden_layer_dims,
                                        output_units=1,
                                        activation=activation,
                                        use_batch_norm=use_batch_norm,
                                        batch_norm_moment=batch_norm_moment,
                                        dropout=dropout)

    # Collect feature names for mapping
    features = preprocess_features()
    feature_cols = np.array(features['cols'])
    
    # Create specs for pipeline
    context_spec_ = {}
    example_spec_ = {feat: tf.io.FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=0.0) for feat in feature_cols}
    label_spec_ = ('relevance_label', tf.io.FixedLenFeature(shape=(1,), dtype=tf.int64, default_value=-1))

    # Log1p preprocessing spec for data normalization
    preprocess_spec = {
        **{name: lambda t: tf.math.log1p(t * tf.sign(t)) * tf.sign(t)
           for name in example_spec_.keys()}
    }

    # Define Input Creator
    input_creator= tfr.keras.model.FeatureSpecInputCreator(
            context_feature_spec={},
            example_feature_spec=example_spec_)
    
    # Define Model
    if preprocessor == False:
        model_builder = tfr.keras.model.ModelBuilder(
            input_creator=input_creator,
            preprocessor=tfr.keras.model.PreprocessorWithSpec(),
            scorer=scorer,
            mask_feature_name="example_list_mask",
            name="model_builder")
    else:
        model_builder = tfr.keras.model.ModelBuilder(
        input_creator=input_creator,
        preprocessor=tfr.keras.model.PreprocessorWithSpec(preprocess_spec),
        scorer=scorer,
        mask_feature_name="example_list_mask",
        name="model_builder")

    # Define Dataset Parameters
    dataset_hparams = tfr.keras.pipeline.DatasetHparams(
        train_input_pattern=train_input_pattern,
        valid_input_pattern=valid_input_pattern,
        train_batch_size=train_batch_size,
        valid_batch_size=valid_batch_size,
        list_size=list_size,
        dataset_reader=tfr.keras.pipeline.DatasetHparams.dataset_reader)

    # Define Dataset Builder
    dataset_builder = tfr.keras.pipeline.SimpleDatasetBuilder(
        {},
        example_spec_,
        mask_feature_name="example_list_mask",
        label_spec=label_spec_,
        hparams=dataset_hparams,
        sample_weight_spec=None)

    # Define Pipeline Hparams
    pipeline_hparams = tfr.keras.pipeline.PipelineHparams(
        model_dir=model_dir,
        num_epochs=num_epochs,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        learning_rate=learning_rate,
        loss=loss,
        optimizer=optimizer)
    
    # Define Ranking Pipeline
    ranking_pipeline = tfr.keras.pipeline.SimplePipeline(
        model_builder,
        dataset_builder=dataset_builder,
        hparams=pipeline_hparams)
    
    # Train and Validate
    ranking_pipeline.train_and_validate(verbose=1)

In [10]:
build_model(preprocessor=True, model_dir='base_model_with_preprocessing')

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2024-06-03 11:26:57,450	INFO worker.py:1752 -- Started a local Ray instance.
2024-06-03 11:27:12.395714: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/5
5000/5000 [==============================] - 3923s 785ms/step - loss: -0.6773 - metric/ndcg_1: 0.4123 - metric/ndcg_5: 0.4071 - metric/ndcg_10: 0.4255 - metric/ndcg: 0.6785 - val_loss: -0.6916 - val_metric/ndcg_1: 0.4433 - val_metric/ndcg_5: 0.4338 - val_metric/ndcg_10: 0.4523 - val_metric/ndcg: 0.6930
Epoch 2/5
5000/5000 [==============================] - 3838s 768ms/step - loss: -0.6884 - metric/ndcg_1: 0.4366 - metric/ndcg_5: 0.4256 - metric/ndcg_10: 0.4434 - metric/ndcg: 0.6891 - val_loss: -0.6944 - val_metric/ndcg_1: 0.4442 - val_metric/ndcg_5: 0.4359 - val_metric/ndcg_10: 0.4567 - val_metric/ndcg: 0.6953
Epoch 3/5
5000/5000 [==============================] - 3832s 766ms/step - loss: -0.6914 - metric/ndcg_1: 0.4431 - metric/ndcg_5: 0.4309 - metric/ndcg_10: 0.4483 - metric/ndcg: 0.6921 - val_loss: -0.6950 - val_metric/ndcg_1: 0.4416 - val_metric/ndcg_5: 0.4380 - val_metric/ndcg_10: 0.4574 - val_metric/ndcg: 0.6957
Epoch 4/5
5000/5000 [==============================] - 371

INFO:tensorflow:Assets written to: base_model_with_preprocessing/export/latest_model/assets


In [11]:
# Build Test Dataset for Model Ingestion

features = preprocess_features()
feature_cols = np.array(features['cols'])

# Create specs for pipeline
context_spec_ = {}
example_spec_ = {feat: tf.io.FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=0.0) for feat in feature_cols}
label_spec_ = ('relevance_label', tf.io.FixedLenFeature(shape=(1,), dtype=tf.int64, default_value=-1))

dataset_hparams = tfr.keras.pipeline.DatasetHparams(
    train_input_pattern='tfrecord_1/train.tfrecords',
    valid_input_pattern='tfrecord_1/test.tfrecords',
    train_batch_size=128,
    valid_batch_size=128,
    list_size=200,
    dataset_reader=tfr.keras.pipeline.DatasetHparams.dataset_reader)

# Define Dataset Builder
dataset_builder = tfr.keras.pipeline.SimpleDatasetBuilder(
    {},
    example_spec_,
    mask_feature_name="example_list_mask",
    label_spec=label_spec_,
    hparams=dataset_hparams,
    sample_weight_spec=None)

ds = dataset_builder.build_valid_dataset()

model_with_preprocessing = tf.keras.models.load_model('base_model_with_preprocessing/export/latest_model/')

loss_with_preprocessing = model_with_preprocessing.evaluate(ds.take(2000))

with_preprocessing = pd.DataFrame(loss_with_preprocessing, columns=['With_Preprocessing'], index=['Loss', 'NDCG_1', 'NDCG_5', 'NDCG_10', 'NDCG']).T

final_results = with_preprocessing

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2024-06-04 10:23:24,173	INFO worker.py:1752 -- Started a local Ray instance.
2024-06-04 10:23:38.478960: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


2000/2000 [==============================] - 449s 221ms/step - loss: -0.6915 - metric/ndcg_1: 0.4384 - metric/ndcg_5: 0.4331 - metric/ndcg_10: 0.4517 - metric/ndcg: 0.6923


In [12]:
final_results

,Loss,NDCG_1,NDCG_5,NDCG_10,NDCG
With_Preprocessing,-0.691515,0.438447,0.433117,0.451722,0.692254


In [15]:
# Evaluate model performance
%reload_ext tensorboard

%tensorboard --logdir base_model_with_preprocessing/

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2024-06-03 18:05:54.156108: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-03 18:05:54.244780: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-03 18:05:54.246942: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 18:05:55.489124: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-06-03 18:05:56.843829: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would l

In [13]:
for x, y in ds.take(1):
    break

In [20]:
print(len(x))

137


In [14]:
score = model_with_preprocessing.predict(x)
print(f"score: {score}")
min_score = tf.reduce_min(score)
scores = tf.where(tf.greater_equal(y, 0.), score, min_score - 1e-5)

4/4 [==============================] - 3s 61ms/step
score: [[ -1.5973206  -15.563038    -7.1940303  ... -23.02585    -23.02585
  -23.02585   ]
 [-16.63225     -0.13893722   1.5003042  ... -23.02585    -23.02585
  -23.02585   ]
 [ -5.431166     3.6945255    0.2204494  ... -23.02585    -23.02585
  -23.02585   ]
 ...
 [  1.852209     2.557881     2.618662   ... -23.02585    -23.02585
  -23.02585   ]
 [ -4.5550756   -9.186165   -19.309807   ... -23.02585    -23.02585
  -23.02585   ]
 [  1.418303    -2.9753964    9.088194   ... -23.02585    -23.02585
  -23.02585   ]]


In [15]:
print(score.shape)

(128, 200)


In [16]:
print(scores)
print(scores.shape)

tf.Tensor(
[[ -1.5973206  -15.563038    -7.1940303  ... -67.37445    -67.37445
  -67.37445   ]
 [-16.63225     -0.13893722   1.5003042  ... -67.37445    -67.37445
  -67.37445   ]
 [ -5.431166     3.6945255    0.2204494  ... -67.37445    -67.37445
  -67.37445   ]
 ...
 [  1.852209     2.557881     2.618662   ... -67.37445    -67.37445
  -67.37445   ]
 [ -4.5550756   -9.186165   -19.309807   ... -67.37445    -67.37445
  -67.37445   ]
 [  1.418303    -2.9753964    9.088194   ... -67.37445    -67.37445
  -67.37445   ]], shape=(128, 200), dtype=float32)
(128, 200)


In [48]:
batch_idx = 0
batch_scores = scores[batch_idx]
batch_feature = x['QualityScore'][batch_idx]
batch_scores = scores[batch_idx:batch_idx+1]  # Shape: [1, list_size]
batch_feature = x['QualityScore'][batch_idx:batch_idx+1]  # Shape: [1, list_size]

In [49]:
print(batch_scores)
print(batch_scores.shape)

tf.Tensor(
[[ -1.5973206  -15.563038    -7.1940303   15.137175   -21.49283
   17.021065    -9.184447    -0.39191926  20.675968     1.6588693
   20.131027   -12.241798    15.923479    39.361       17.02776
   13.912825    19.912148    24.841759    16.006718    -9.9095545
  -30.8385      11.960856    12.485785     0.49007228   2.3352404
    1.0140514  -16.398066     7.9764      20.135534    11.439901
   16.796497     3.9551904   -2.1022117    2.0299447   42.725243
    6.2371097   -0.7471613   27.294209    25.002216   -15.886215
    1.1301229    3.0540493  -10.876614   -14.174013    11.431093
  -11.374411   -17.353363    49.800896    24.909369    -9.049595
    8.359512     8.777021     2.380428     2.5781476   24.576576
   61.554        1.967894    32.933594    27.445902   -33.458908
   27.565468    12.552557    19.824291    44.633293    16.072903
    1.2291312   12.370058    11.935087     2.7311378    8.730278
   -1.5253975   18.925201    -8.471321    20.274052     1.8754278
    7.230947

In [58]:
tf.argmax(batch_scores, 1)


<tf.Tensor: shape=(1,), dtype=int64, numpy=array([55])>

In [50]:
print(batch_feature)
print(batch_feature.shape)

tf.Tensor(
[[[ 28.]
  [ 65.]
  [112.]
  [  1.]
  [ 15.]
  [  1.]
  [  3.]
  [  1.]
  [ 16.]
  [129.]
  [ 15.]
  [  5.]
  [  7.]
  [ 16.]
  [  5.]
  [ 48.]
  [  8.]
  [ 13.]
  [  2.]
  [  9.]
  [  6.]
  [ 12.]
  [  6.]
  [  1.]
  [  3.]
  [132.]
  [  2.]
  [  7.]
  [ 25.]
  [ 21.]
  [185.]
  [  6.]
  [  1.]
  [  9.]
  [ 18.]
  [116.]
  [  4.]
  [  2.]
  [ 40.]
  [ 69.]
  [ 79.]
  [  7.]
  [ 14.]
  [ 27.]
  [  4.]
  [ 52.]
  [ 28.]
  [ 16.]
  [ 97.]
  [ 15.]
  [  6.]
  [  9.]
  [ 20.]
  [ 58.]
  [  1.]
  [ 27.]
  [ 77.]
  [ 21.]
  [ 15.]
  [ 51.]
  [  7.]
  [  1.]
  [ 32.]
  [  1.]
  [ 16.]
  [ 15.]
  [ 31.]
  [ 15.]
  [  1.]
  [156.]
  [ 14.]
  [  4.]
  [  6.]
  [  2.]
  [ 33.]
  [ 31.]
  [ 33.]
  [  5.]
  [ 19.]
  [  1.]
  [ 39.]
  [  1.]
  [  9.]
  [ 34.]
  [  1.]
  [ 29.]
  [ 20.]
  [248.]
  [ 31.]
  [  6.]
  [  1.]
  [  1.]
  [ 12.]
  [  1.]
  [ 14.]
  [  4.]
  [  9.]
  [ 34.]
  [  2.]
  [ 10.]
  [  5.]
  [  7.]
  [ 10.]
  [  1.]
  [ 21.]
  [  8.]
  [  4.]
  [  7.]
  [ 44.]
  [ 22.]

In [59]:
print(batch_feature[0][55][0])

tf.Tensor(27.0, shape=(), dtype=float32)


In [56]:
# Sort by scores
sorted_indices = tfr.utils.sort_by_scores(
    batch_scores,
    [batch_feature]
)[0]  # We only need the sorted indices

# Collect sorted answers
sorted_answers = []
sorted_answers.append(sorted_indices)

# Convert list of sorted answers to numpy array
sorted_answers = np.array(sorted_answers)
sorted_answers = np.reshape(sorted_answers, (-1,))
print(f"Sorted answers: {sorted_answers}")

print(sorted_answers.shape)

Sorted answers: [ 27.  34.  16.   8.   2.   1.   6.   2.  18.  30.  33.  33.  20.  16.
  31.  14.  29.   9.  21.   3.   2.   7.  15.   2.   4.  21.  40.  97.
  13.   1.  12.  16.  12.   9.   2.  25.  15.   8. 140.  32.  39.   4.
   1.   2.   5.   1. 185.  16.   2.  10.   7.  10.   6.   1.   5.  48.
  14.   1.   6.  31.  12.  15.  21.   4.   1.   7. 248.  21.   9. 156.
   6.   7.   6.  65.  31. 116.   1.  13.  19.   6.  14.   5.  27.   7.
   9.   1.  58.  20.   3.   9.   9.  77.  33. 129.  22.  15.  79. 132.
  29.  34.   1.  13.   4.   1.   4.  14.  28.   1.   5.   1.   8.  31.
   1. 112.   6.  44.  15.   3.   9.  14.  52.   5.  27.   3.  65.   4.
  69.   2.  28.   3.  15.   1.   1.   7.   6.   3.  51.   1.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 

In [67]:
keysList = list(x.keys())
print(keysList)
print(len(keysList))
print(y)

['BM25_anchor', 'BM25_body', 'BM25_title', 'BM25_url', 'BM25_whole_document', 'IDF_Inverse_document_frequency__anchor', 'IDF_Inverse_document_frequency__body', 'IDF_Inverse_document_frequency__title', 'IDF_Inverse_document_frequency__url', 'IDF_Inverse_document_frequency__whole_document', 'Inlink_number', 'LMIR.ABS_anchor', 'LMIR.ABS_body', 'LMIR.ABS_title', 'LMIR.ABS_url', 'LMIR.ABS_whole_document', 'LMIR.DIR_anchor', 'LMIR.DIR_body', 'LMIR.DIR_title', 'LMIR.DIR_url', 'LMIR.DIR_whole_document', 'LMIR.JM_anchor', 'LMIR.JM_body', 'LMIR.JM_title', 'LMIR.JM_url', 'LMIR.JM_whole_document', 'Length_of_URL', 'Number_of_slash_in_URL', 'Outlink_number', 'PageRank', 'QualityScore', 'QualityScore2', 'Query-url_click_count', 'SiteRank', 'boolean_model_anchor', 'boolean_model_body', 'boolean_model_title', 'boolean_model_url', 'boolean_model_whole_document', 'covered_query_term_number_anchor', 'covered_query_term_number_body', 'covered_query_term_number_title', 'covered_query_term_number_url', 'cov

In [ ]:
for batch_idx in range(scores.shape[0]):  # Assuming scores has shape (num_batches, list_size)
    batch_scores = scores[batch_idx]
    batch_feature = x['covered_query_term_number_body'][batch_idx]
    
    # Sort by scores
    sorted_indices = tfr.utils.sort_by_scores(
        batch_scores,
        batch_feature
    )[0]  # We only need the sorted indices

    # Collect sorted answers
    sorted_answers.append(sorted_indices)

# Convert list of sorted answers to numpy array
sorted_answers = np.array(sorted_answers)

print(f"Sorted answers: {sorted_answers}")

In [17]:
# Predict scores on the testing data
predictions = model_with_preprocessing.predict(ds.take(2000))

# Convert predictions to a pandas DataFrame if needed
predictions_df = pd.DataFrame(predictions, columns=['Score'])

# Display the predictions
print(predictions_df)

2000/2000 [==============================] - 394s 195ms/step


ValueError: Shape of passed values is (256000, 200), indices imply (256000, 1)

In [20]:
print(predictions)
print(predictions.shape)

[[ -1.5973206  -15.563038    -7.1940303  ... -23.02585    -23.02585
  -23.02585   ]
 [-16.63225     -0.13893722   1.5003042  ... -23.02585    -23.02585
  -23.02585   ]
 [ -5.431166     3.6945255    0.2204494  ... -23.02585    -23.02585
  -23.02585   ]
 ...
 [-10.578133   -12.728604   -13.967123   ... -23.02585    -23.02585
  -23.02585   ]
 [  1.5879908    3.416619     8.806359   ... -23.02585    -23.02585
  -23.02585   ]
 [-21.300102   -12.410904   -15.983486   ... -23.02585    -23.02585
  -23.02585   ]]
(256000, 200)
